<a href="https://colab.research.google.com/github/nyp-sit/mindef-ai/blob/main/day1-pm/video-anomaly-exercise-v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://www.nyp.edu.sg/content/dam/nyp/logo.png" width="238" height="70"/>

Welcome to the lab! Before we get started here are a few pointers on Jupyter notebooks.

1. The notebook is composed of cells; cells can contain code which you can run, or they can hold text and/or images which are there for you to read.

2. You can execute code cells by clicking the ```Run``` icon in the menu, or via the following keyboard shortcuts ```Shift-Enter``` (run and advance) or ```Ctrl-Enter``` (run and stay in the current cell).

3. To interrupt cell execution, click the ```Stop``` button on the toolbar or navigate to the ```Kernel``` menu, and select ```Interrupt ```.
    

In [1]:
!wget -q https://nyp-aicourse.s3-ap-southeast-1.amazonaws.com/codes/dataset_util.py

'wget' is not recognized as an internal or external command,
operable program or batch file.


# Video Anomaly Detection (Exercise)
                                                             
Using the previous lab *Video Anomaly Detection* as reference, try to train the model on another dataset and see how it performs.  Complete the codes marked with *TODO*.

## Import libraries

In [ ]:
import tensorflow as tf
import os
from utils import *
from IPython.display import display
from IPython.display import Image as ipyImage
from dataset_util import prepare_dataset

## Dataset 


In [ ]:
base_dataset_dir = 'video_dataset'

**Exercise**

Complete the code below to use the dataset 'UCSDped2'.

<details><summary>Click here for answer</summary>
<br/>
    
```
dataset = 'UCSDped2'
```
</details>

In [ ]:
# TODO:  modify the code
dataset =  None

# setup all the relative path
root_path = os.path.join(base_dataset_dir, dataset)
train_dir = os.path.join(root_path, 'Train')
test_dir = os.path.join(root_path, 'Test')

### Visualize the Train dataset

**Exercise**

Complete the codes below to display sample images from the train sample folder "Train003". 

<details><summary>Click here for answer</summary>
<br/>
    
```
train_sample_folder = 'Train003' 
image_range = (1,9)  # this display image from 1 to 8
image_folder = os.path.join(train_dir, train_sample_folder)
display_images(image_folder, image_range=image_range, max_per_row=4) 
```
</details>

In [ ]:
# TODO: complete the code 

train_sample_folder = None
image_range = None  # this display image from 10 to 18
image_folder = None
display_images(image_folder, image_range=image_range, max_per_row=4)

We now visualize the image frames as animated gif (like a video)

In [ ]:
gif_filename = train_sample_folder + '.gif' 
create_gif(image_folder, gif_filename, img_type='tif')

with open(gif_filename,'rb') as file:
    display(ipyImage(file.read(), format='png'))

## Visualize the Test dataset

**Exercise**

- Display test sample images from folder "Test001"
- Call the create_gif() function to create an animated gif of the video frames in the 'Test001' folder. 
- Display the gif file.

<details><summary>Click here for answer</summary>
<br/>
    
```
test_sample_folder = 'Test001' 
image_range = (1,9)  # this display image from 1 to 8
image_folder = os.path.join(test_dir, test_sample_folder)
display_images(image_folder, image_range=image_range, max_per_row=4)

gif_filename = train_sample_folder + '.gif' 
create_gif(image_folder, gif_filename, img_type='tif')

with open(gif_filename,'rb') as file:
    display(ipyImage(file.read(), format='png'))
    
```
</details>

In [ ]:
## TODO: Write the coder here, refering to previous lab




### Prepare Training Dataset

Here we create a Tensorflow dataset suitable for use in training the Autoencoder network later.

In [ ]:
IMG_HEIGHT=100
IMG_WIDTH=100
BATCH_SIZE=16

train_fileset = os.path.join(train_dir, '*/*.tif')


train_dataset, test_dataset = prepare_dataset(train_fileset,
                                img_height=IMG_HEIGHT, 
                                img_width=IMG_WIDTH, 
                                batch_size=BATCH_SIZE,
                                shuffle=True,
                                split=True,
                                test_size=0.2)

### Building the Auto-encoder Model

Here we build our autoencoder network.  

**Exercise**

Change the latent layer's dimension to 1500.


<details><summary>Click here for answer</summary>
<br/>
    
```
encoded = tf.keras.layers.Dense(2000)(x)
```
    
</details>

In [ ]:
# The encoder part of the Audo-encoder model
inputs = tf.keras.layers.Input(shape=(100,100,1))
x = tf.keras.layers.Conv2D(32, kernel_size=5, activation='relu')(inputs)
x = tf.keras.layers.MaxPool2D(pool_size=2)(x)
x = tf.keras.layers.Conv2D(32, kernel_size=5, activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=2)(x)
x = tf.keras.layers.Flatten()(x)

## TODO: Modif the code 
encoded = tf.keras.layers.Dense(2000)(x)


encoder = tf.keras.Model(inputs=[inputs], outputs=[encoded])

# The decoder part of the Audo-encoder model
decoder_inputs = tf.keras.layers.Input(shape=(2000))
x = tf.keras.layers.Dense(22*22*32, activation='relu')(decoder_inputs)
x = tf.keras.layers.Reshape(target_shape=(22,22,32))(x)
x = tf.keras.layers.UpSampling2D(2, interpolation='nearest')(x)
x = tf.keras.layers.Conv2DTranspose(32, kernel_size=5, activation='relu')(x)
x = tf.keras.layers.UpSampling2D(2, interpolation='nearest')(x)
decoded = tf.keras.layers.Conv2DTranspose(1, kernel_size=5, activation='sigmoid')(x)
decoder = tf.keras.Model(inputs=[decoder_inputs], outputs=[decoded])

encoding = encoder(inputs)
decoding = decoder(encoding)
conv_ae = tf.keras.Model(inputs=[inputs], outputs=[decoding])

In [ ]:
conv_ae.compile(loss=tf.keras.losses.MeanSquaredError(), 
        optimizer=tf.keras.optimizers.Adam(lr=1e-4, decay=1e-4),
        metrics=['mae'])

**Exercise**

Train the network for 40 epochs.

<details><summary>Click here for answer</summary>
<br/>
    
```
num_epochs = 40

history = conv_ae.fit(train_dataset, 
                      validation_data=test_dataset,
                      epochs=num_epochs)
```
</details>

In [ ]:
## TODO: Complete the code 

num_epochs = None

history = None

In [ ]:
plot_training_loss(history.history)

### Prepare Testing Dataset

Now we will create a test dataset that we can use to test the trained auto-encoder. 

**Exercise**:

Add code below to prepare a dataset using samples from test folder "Test002".

<details><summary>Click here for answer</summary>
<br/>
    
```
BATCH_SIZE=1

test_sample_folder = 'Test002'

test_fileset = os.path.join(test_dir, test_sample_folder, "*.tif")

test_dataset = prepare_dataset(test_fileset,
                                img_height=IMG_HEIGHT, 
                                img_width=IMG_WIDTH, 
                                batch_size=BATCH_SIZE,
                                shuffle=False)
```
</details>

In [ ]:
## TODO: Complete the code 

BATCH_SIZE=1

test_sample_folder = None 

test_fileset = os.path.join(test_dir, test_sample_folder, "*.tif")

test_dataset = None 


#### Reconstruction loss over different video frames

Let us show the reconstruction loss animated over different video frames

In [ ]:
create_losses_animation(conv_ae, test_dataset, "losses.gif")
with open('losses.gif','rb') as file:
    display(ipyImage(file.read(), format='png'))

#### Identification of anomalous object from the video frames

**Exercise**

Identify and highlight the area of anomaly in the video frame, using a threshold of 3.5.

<details><summary>Click here for answer</summary>
<br/>

```
threshold = 4.0
identify_anomaly(conv_ae, test_dataset, "video.gif", threshold)
with open('video.gif','rb') as file:
    display(ipyImage(file.read(), format='png'))
```
</details>

In [ ]:
## TODO: Complete the code 